<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/16_Finans_i%C3%A7in_Python_MACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphanelerin yüklenmesi
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener
!pip install backtesting

#kütüphanelerin çağırılması
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy

#Tradingview Kütüphanesini açıyoruz.
tv = TvDatafeed()

#EMA Fonksiyonu
def ema(seri, uzunluk):
    """
    Verilen bir seri için Üssel Hareketli Ortalama (EMA) hesaplayın.
    """
    return seri.ewm(span=uzunluk, adjust=False).mean()

#MACD Fonksiyonu
def macd(seri, hizli=12, yavas=26, sinyal=9):
    """
    Verilen bir seri için Hareketli Ortalama Yakınsama Iraksama (MACD) hesaplayın ve bir DataFrame döndürün.

    Parametreler:
    - seri: Fiyat verilerini içeren pandas Serisi.
    - hizli (int): Hızlı EMA için periyot. Varsayılan 12'dir.
    - yavas (int): Yavaş EMA için periyot. Varsayılan 26'dır.
    - sinyal (int): Sinyal çizgisi EMA'sı için periyot. Varsayılan 9'dur.

    """
    ema_hizli = ema(seri, hizli)
    ema_yavas = ema(seri, yavas)
    macd_cizgisi = ema_hizli - ema_yavas
    sinyal_cizgisi = ema(macd_cizgisi, sinyal)
    macd_histogram = macd_cizgisi - sinyal_cizgisi

    return macd_cizgisi,sinyal_cizgisi, macd_histogram

#
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

Hisse = "THYAO"
data = tv.get_hist(symbol=Hisse, exchange='BIST', interval=Interval.in_daily, n_bars=5000)
data['MACD'],data['Signal'],data['Histogram'] = macd(data['close'],12,26,9)
print(data)

data['Entry'] = data['MACD'] > data['Signal']
data['Exit'] = data['MACD'] < data['Signal']


data.rename(columns={'open': 'Open', 'high': 'High',
                     'low': 'Low',
                     'close': 'Close',
                     'volume': 'Volume'},
                     inplace=True)


# Backtest simulatörünü çalıştırma
bt = Backtest(data, Strategy, cash=100000, commission=0.002)
Stats = bt.run()
print(Stats)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-uxrnd0mk
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-uxrnd0mk
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


                         symbol        open        high         low  \
datetime                                                              
2004-08-13 06:00:00  BIST:THYAO    0.919349    0.938370    0.913008   
2004-08-16 06:00:00  BIST:THYAO    0.925689    0.925689    0.913008   
2004-08-17 06:00:00  BIST:THYAO    0.925689    0.938370    0.919349   
2004-08-18 06:00:00  BIST:THYAO    0.938370    0.938370    0.919349   
2004-08-19 06:00:00  BIST:THYAO    0.925689    0.938370    0.906668   
...                         ...         ...         ...         ...   
2024-07-30 06:00:00  BIST:THYAO  294.500000  297.000000  292.500000   
2024-07-31 06:00:00  BIST:THYAO  291.250000  292.000000  288.250000   
2024-08-01 06:00:00  BIST:THYAO  293.000000  298.250000  293.000000   
2024-08-02 06:00:00  BIST:THYAO  292.500000  293.750000  288.250000   
2024-08-05 06:00:00  BIST:THYAO  271.250000  280.750000  271.250000   

                          close      volume  
datetime                      